## Problem statement

Let $\Phi \in \mathbb{R}^{n\times n}$ and let $\Omega_0 = \langle c_0, G_0\rangle$ be a zonotope, i.e. a set defined by its center $c \in \mathbb{R}^n$ and generators matrix $G_0 \in \mathbb{R}^{n\times p}$:

$$\Omega_0 = \left\{ x \in \mathbb{R}^n: x = c + \sum_{i=1}^p \xi_i g_i,\qquad  \xi_i \in [-1, 1]~~\forall i \in 1,\ldots, p\right\},
$$
where $g_i$ denotes the $i$-th column of $G$.

How do you go about efficiently computing $\{\Omega_0, \Phi \Omega_0, \ldots, \Phi^{k-1}\Omega_0\}$ in Julia?

## Running example

Our running example is the "motor" model.

In [ ]:
Φ, Ω0, N

In [7]:
using Revise, LazySets, BenchmarkTools, Test, StaticArrays
using LinearAlgebra

In [3]:
Φ = rand(2, 2)
Ω0 = rand(Zonotope, dim=2);

In [ ]:
function seq_v1(Φ::AbstractMatrix{N}, Ω0::Zonotope{N, VN, MN}, kmax::Int) where {N, VN, MN}
    c0 = Ω0.center
    G0 = Ω0.generators

    out = Vector{typeof(Ω0)}(undef, kmax)
    out[1] = Ω0

    Φ_power_k = copy(Φ)
    Φ_power_k_cache = similar(Φ)

    @inbounds for i in 1:kmax-1
        mul!(ΦV0mat, Φ_power_k, V0.vertices)
        push!(res, VPolygonM(copy(ΦV0mat)))

        mul!(Φ_power_k_cache, Φ_power_k, Φ)
        copy!(Φ_power_k, Φ_power_k_cache)
    end
    return res
end

In [ ]:
function seq_v2(Φ::AbstractMatrix{N}, Ω0::Zonotope{N, VN, MN}, kmax::Int) where {N, VN, MN}
    c0 = Ω0.center
    G0 = Ω0.generators

    out = Vector{typeof(Ω0)}(undef, kmax)
    out[1] = Ω0

    Φ_power_k = copy(Φ)
    Φ_power_k_cache = similar(Φ)

    @inbounds for i in 1:kmax-1
        mul!(ΦV0mat, Φ_power_k, V0.vertices)
        push!(res, VPolygonM(copy(ΦV0mat)))

        mul!(Φ_power_k_cache, Φ_power_k, Φ)
        copy!(Φ_power_k, Φ_power_k_cache)
    end
    return res
end

In [9]:
?mul!

search: mul! rmul! lmul! accumulate! muladd widemul accumulate module Module



```
mul!(Y, A, B) -> Y
```

Calculates the matrix-matrix or matrix-vector product $AB$ and stores the result in `Y`, overwriting the existing value of `Y`. Note that `Y` must not be aliased with either `A` or `B`.

# Examples

```jldoctest
julia> A=[1.0 2.0; 3.0 4.0]; B=[1.0 1.0; 1.0 1.0]; Y = similar(B); mul!(Y, A, B);

julia> Y
2×2 Array{Float64,2}:
 3.0  3.0
 7.0  7.0
```

# Implementation

For custom matrix and vector types, it is recommended to implement 5-argument `mul!` rather than implementing 3-argument `mul!` directly if possible.

---

```
mul!(C, A, B, α, β) -> C
```

Combined inplace matrix-matrix or matrix-vector multiply-add $A B α + C β$. The result is stored in `C` by overwriting it.  Note that `C` must not be aliased with either `A` or `B`.

!!! compat "Julia 1.3"
    Five-argument `mul!` requires at least Julia 1.3.


# Examples

```jldoctest
julia> A=[1.0 2.0; 3.0 4.0]; B=[1.0 1.0; 1.0 1.0]; C=[1.0 2.0; 3.0 4.0];

julia> mul!(C, A, B, 100.0, 10.0) === C
true

julia> C
2×2 Array{Float64,2}:
 310.0  320.0
 730.0  740.0
```
